### Importing packages

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


###  Load the pretrained Network

In [2]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

###  Freeze the layers

In [4]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

###  Training parameters

In [5]:
img_width, img_height = 150, 150
train_data_dir = 'Train'
val_data_dir = 'Test'
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 4
nb_val_samples = 4
nb_epochs = 20

###  Build a classification model on top of Base Network

In [6]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(3, activation='softmax')(x)
model = Model(input=input, output=x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                524352    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 15,239,235
Trainable params: 524,547
Non-trainable params: 14,714,688
_________________________________________________________________


###  Dataset Preparation

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 26 images belonging to 3 classes.
Found 7 images belonging to 3 classes.


###  Training

In [9]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, validation_data=validation_generator, nb_val_samples=nb_val_samples)

print('Training Completed!')

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1/1 [==============================] - 9s 9s/step - loss: 1.0717 - acc: 0.2500 - val_loss: 4.6864 - val_acc: 0.4286
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 1.9419 - acc: 0.7500 - val_loss: 5.3634 - val_acc: 0.2857
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 8.3151 - acc: 0.2500 - val_loss: 1.5613 - val_acc: 0.4286
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 2.0155 - acc: 0.5000 - val_loss: 0.4839 - val_acc: 0.7143
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 0.6330 - acc: 0.7500 - val_loss: 2.6907 - val_acc: 0.2857
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 2.5098 - acc: 0.2500 - val_loss: 0.6984 - val_acc: 0.7143
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 2.4492 - acc: 0.5000 - val_loss: 0.7399 - val_acc: 0.5714
Epoch 8/20
1/1 [==============================] - 3s 3s/step - 

###  Test the model

In [10]:
img_path = 'strangle.jpg'
label = ['fall','kick','strangle']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features >= 0.9)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

Predicted Array: [[2.4563029e-14 1.3518772e-16 1.0000000e+00]]
Predicted Label: strangle


In [11]:
img_path = 'kick.jpg'
label = ['fall','kick','strangle']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features >= 0.9)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

Predicted Array: [[2.2359221e-29 1.0000000e+00 1.9426279e-26]]
Predicted Label: kick


In [12]:
img_path = 'fall.jpg'
label = ['fall','kick','strangle']
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
ind = np.where(features >= 0.9)[1][0]
print('Predicted Array:',features)
print('Predicted Label:',label[ind])

Predicted Array: [[1.0000000e+00 4.6733706e-17 4.4210206e-29]]
Predicted Label: fall


In [ ]:
model = Sequential()

### Models for image classification with weights trained on ImageNet
* Xception
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* MobileNet
* DenseNet
* NASNet